In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import sqlite3
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
# 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
driver = webdriver.Chrome('C:/Users/noilkwon/dev/기타/chromedriver.exe', chrome_options=options)

In [3]:
# DB 생성
conn = sqlite3.connect('auction.db')
cur = conn.cursor()

In [4]:
# 테이블 생성
#cur.execute('create table 경매(사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수)')
#cur.execute('drop table 경매')
cur.execute('delete from 경매')

In [5]:
# 홈페이지 접속
url = r'http://www.courtauction.co.kr/maemul/search_total.asp'
driver.get(url)

In [6]:
# 매각기일
selector = ['table#Table1','tbody','tr:nth-of-type(2)','td','#Table4','tbody','tr','td:nth-of-type(2)','input#sdate']
selector = ' > '.join(selector)
driver.find_element_by_css_selector(selector).clear()
driver.find_element_by_css_selector(selector).send_keys('2000-01-01')

In [7]:
# 소재지
select = Select(driver.find_element_by_id('sido'))
select.select_by_visible_text('서울')

In [8]:
# 용도 복수선택
driver.find_element_by_id('Checkbox8').click()

In [9]:
# 정렬방식
select = Select(driver.find_element_by_id('sort'))
select.select_by_value('1')

In [10]:
# 버튼 클릭
driver.find_element_by_id('search').click()

In [ ]:
# 함수 정의
def getText(row,col):
    selector = ['#Table22','tbody','tr:nth-of-type('+str(row)+')','td:nth-of-type('+str(col)+')']
    selector = ' > '.join(selector)
    text = driver.find_element_by_css_selector(selector).text
    return text

def getPage():
    data = []
    for r in np.arange(6,45,2):
        values = []
        # 사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수
        for c in [2,4,5,6,7,8]:   
            values.append(getText(r,c))
        data.append(values)
    return data

In [ ]:
# 작업 시작
page = 730

for i in range(page):
    j = i//10
    driver.execute_script("GoPage("+str(j)+","+str(i+1)+",'board')")
    cur.executemany("insert into 경매 values (?,?,?,?,?,?)", getPage())
    if i%10 == 0:
        conn.commit()
        print("{}번째 페이지 적재".format(i+1))

1번째 페이지 적재


In [ ]:
# 적재 데이터 출력
cur.execute("select * from 경매")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['번호', '소재지', '가격', '매각기일', '진행상태', '조회수'])

In [ ]:
# 중복 데이터 제거
u, indices = np.unique(df['번호'], return_index=True)
indices.sort()
df = df.iloc[indices,:]

In [ ]:
# 번호 분리
df['법원'] = df['번호'].str.split('\n').str.get(0)
df['사건번호'] = df['번호'].str.split('\n').str.get(1)
df['용도'] = df['번호'].str.split('\n').str.get(2)

In [ ]:
# 가격 분리 
df['감정가'] = df['가격'].str.split('\n').str.get(0)
df['최저입찰가'] = df['가격'].str.split('\n').str.get(1)
df['최저입찰가율'] = df['가격'].str.split('\n').str.get(2)

# 형(Type) 변환
df['최저입찰가'] = df['최저입찰가'].str.replace(',','').astype(float)
df['감정가'] = df['감정가'].str.replace(',','').astype(float)
df['최저입찰가율'] = df['최저입찰가율'].str.replace('%','').astype(float)

In [ ]:
# 매각기일 추출
df['매각일'] = df['매각기일'].str.split('\n').str.get(0)

In [ ]:
# 소재지 추출
df['주소'] = df['소재지'].str.split('\n').str.get(0)
df['면적'] = df['소재지'].str.split('\n').str.get(1)

In [ ]:
# 기타정보 추출
# 건물이랑 토지 바뀌어서 들어가는 거 처리해야 함
# 문자에서 숫자만 추출해야 함
df['건물'] = df['면적'].str.split(' ').str.get(0)
df['토지'] = df['면적'].str.split(' ').str.get(1)
df['키1'] = df['면적'].str.split(' ').str.get(2)
df['키2'] = df['면적'].str.split(' ').str.get(3)
df['임시1'] = df['면적'].str.split(' ').str.get(4)
df['임시2'] = df['면적'].str.split(' ').str.get(5)
df.loc[df['임시1'].notnull(),['키1','키2']] = df.loc[df['임시1'].notnull(),['임시1','임시2']]
df.loc[df['키1']=='','키1'] = np.nan
df.loc[df['키2']=='','키2'] = np.nan

In [ ]:
# 진행상태 추출
df['진행'] = df['진행상태'].str.split('\n').str.get(0)
df['유찰횟수'] = df['진행상태'].str.split('\n').str.get(1).str.extract('유찰(\d)회', expand=True)
df.loc[(df['유찰횟수'].isna()) & (df['진행']=='진행'),'유찰횟수'] = 0

In [ ]:
df = df[['법원','사건번호','용도','감정가','최저입찰가','매각일','주소','건물','토지','키1','키2','진행','유찰횟수']]
df.head()